In [1]:
from numpy.random import RandomState
import pandas as pd
import jieba
import jieba.analyse
import re

In [2]:
import csv
with open('movies1.csv','rb') as csvfile:
    reader = csv.DictReader(csvfile)
    column = [row['category'] for row in reader]
print(column)

Error: iterator should return strings, not bytes (did you open the file in text mode?)